In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/big-data-analytics-certification/t2-2-y_train.csv
/kaggle/input/big-data-analytics-certification/t1-data1.csv
/kaggle/input/big-data-analytics-certification/t2-2-X_train.csv
/kaggle/input/big-data-analytics-certification/t2-1-sample_submission.csv
/kaggle/input/big-data-analytics-certification/t1-data2.csv
/kaggle/input/big-data-analytics-certification/t2-1-test.csv
/kaggle/input/big-data-analytics-certification/t2-2-X_test.csv
/kaggle/input/big-data-analytics-certification/t2-1-train.csv


In [3]:
# 라이브러리
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings('ignore')

# 데이터 불러오기
X_train = pd.read_csv('/kaggle/input/big-data-analytics-certification/t2-2-X_train.csv')
X_test = pd.read_csv('/kaggle/input/big-data-analytics-certification/t2-2-X_test.csv')
y_train = pd.read_csv('/kaggle/input/big-data-analytics-certification/t2-2-y_train.csv')

In [4]:
# EDA를 통해 데이터 확인
print(X_train.head())
print(X_test.head())
print(y_train.head())

# shape
print(X_train.shape)
print(X_test.shape)
print(y_train.shape)

# info
print(X_train.info())
print(X_test.info())
print(y_train.info())
# null값은 없음
# 전부 변수형임
# Serial No.은 삭제할 예정

# descirbe
print(X_train.describe())
print('-'*80)
print(X_test.describe())
print('-'*80)
print(y_train.describe())
print('-'*80)

   Serial No.  GRE Score  TOEFL Score  University Rating  SOP  LOR   CGPA  \
0         207        315           99                  2  3.5   3.0  7.89   
1          14        307          109                  3  4.0   3.0  8.00   
2         119        296           99                  2  3.0   3.5  7.28   
3         243        324          115                  3  3.5   3.0  8.76   
4         236        326          111                  5  4.5   4.0  9.23   

   Research  
0         0  
1         1  
2         0  
3         1  
4         1  
   Serial No.  GRE Score  TOEFL Score  University Rating  SOP  LOR   CGPA  \
0         128        319          112                  3  2.5   2.0  8.71   
1          76        329          114                  2  2.0   4.0  8.56   
2         151        334          114                  4  4.0   4.0  9.43   
3         102        312          105                  2  2.5   3.0  8.12   
4         272        299           96                  2  1.5   2.0 

In [5]:
# 전처리
# 불필요 변수 삭제
X_test_ID = X_test['Serial No.']
X_train.drop(columns='Serial No.', inplace=True)
X_test.drop(columns='Serial No.', inplace=True)
y_train.drop(columns='Serial No.', inplace=True)

# 표준화
from sklearn.preprocessing import StandardScaler
Scaler = StandardScaler()
NUM_COL = X_train.columns
X_train[NUM_COL] = Scaler.fit_transform(X_train[NUM_COL])
X_test[NUM_COL] = Scaler.fit_transform(X_test[NUM_COL])

In [6]:
# 데이터 분리
from sklearn.model_selection import train_test_split
X_tr, X_val, y_tr, y_val = train_test_split(X_train, y_train, test_size=0.3)

In [8]:
# ML 모델링
# 선형회귀분석
from sklearn.linear_model import LinearRegression
modelLR = LinearRegression()
modelLR.fit(X_tr, y_tr)

# 릿지 회귀분석
from sklearn.linear_model import Ridge
modelRI = Ridge(alpha=10)
modelRI.fit(X_tr, y_tr)

# 랜덤포레스트 회귀
from sklearn.ensemble import RandomForestRegressor
modelRF = RandomForestRegressor()
modelRF.fit(X_tr, y_tr)

# SVR
from sklearn.svm import SVR
modelSVR = SVR()
modelSVR.fit(X_tr, y_tr)

# XGBoost
from xgboost import XGBRegressor
modelXGB = XGBRegressor()
modelXGB.fit(X_tr, y_tr)

XGBRegressor(base_score=0.5, booster='gbtree', callbacks=None,
             colsample_bylevel=1, colsample_bynode=1, colsample_bytree=1,
             early_stopping_rounds=None, enable_categorical=False,
             eval_metric=None, gamma=0, gpu_id=-1, grow_policy='depthwise',
             importance_type=None, interaction_constraints='',
             learning_rate=0.300000012, max_bin=256, max_cat_to_onehot=4,
             max_delta_step=0, max_depth=6, max_leaves=0, min_child_weight=1,
             missing=nan, monotone_constraints='()', n_estimators=100, n_jobs=0,
             num_parallel_tree=1, predictor='auto', random_state=0, reg_alpha=0,
             reg_lambda=1, ...)

In [9]:
# 각 모델 예측
y_pred_LR = modelLR.predict(X_val)
y_pred_RI = modelRI.predict(X_val)
y_pred_RF = modelRF.predict(X_val)
y_pred_SVR = modelSVR.predict(X_val)
y_pred_XGB = modelXGB.predict(X_val)

In [10]:
# 평가(1차 모델 선정)
from sklearn.metrics import r2_score
print('선형 회귀분석 결정계수 : {:.3f}'.format(r2_score(y_val, y_pred_LR)))
print('Ridge 회귀분석 결정계수 : {:.3f}'.format(r2_score(y_val, y_pred_RI)))
print('랜덤 포레스트 결정계수 : {:.3f}'.format(r2_score(y_val, y_pred_RF)))
print('서포트 벡터 머신 결정계수 : {:.3f}'.format(r2_score(y_val, y_pred_SVR)))
print('XGBoost 결정계수 : {:.3f}'.format(r2_score(y_val, y_pred_XGB)))


선형 회귀분석 결정계수 : 0.824
Ridge 회귀분석 결정계수 : 0.826
랜덤 포레스트 결정계수 : 0.771
서포트 벡터 머신 결정계수 : 0.648
XGBoost 결정계수 : 0.744


In [11]:
# 결정계수가 제일 높은 릿지에 대해서 하이퍼 파라미터를 조정.
from sklearn.model_selection import GridSearchCV
param = {
    'alpha':[5,5.5,6,6.5,7,7.5],
    'max_iter':[9.5,10,10.5,11,11.5,12]
}
# n_jobs 매개 변수를 사용하여 사용할 코어 수를 지정할 수 있다. 사용하는 CPU 코어 개수에 비례해서 속도도 빨라진다.
# n_jobs = -1 (모든 코어 사용)

gs = GridSearchCV(estimator=Ridge(), param_grid=param, scoring='r2', n_jobs=-1, cv=7)
result = gs.fit(X_tr, y_tr)
print('최적 점수 : {:.3f}'.format(result.best_score_))
print('최적 파라미터 : {}'.format(result.best_params_))

modelfinal = Ridge(alpha=6, max_iter=11)
modelfinal.fit(X_tr, y_tr)
y_pred_final = modelfinal.predict(X_val)
print('하이퍼파라미터 릿지 : {:.3f}'.format(r2_score(y_val, y_pred_final)))

최적 점수 : 0.767
최적 파라미터 : {'alpha': 7.5, 'max_iter': 9.5}
하이퍼파라미터 릿지 : 0.826


In [15]:
# 제출
pred=modelfinal.predict(X_test)
pd.DataFrame({'id':X_test_ID, 'target':pred.ravel()}).to_csv('submit.csv', index=False)
# ravel() : 2차원 배열을 1차원으로 펴준다.

# 제출 파일 확인
print(pd.read_csv('submit.csv'))

     id    target
0   128  0.719608
1    76  0.758493
2   151  0.870213
3   102  0.609246
4   272  0.514280
..  ...       ...
75  252  0.686921
76  362  0.878199
77  338  0.908284
78  186  0.829053
79  335  0.725720

[80 rows x 2 columns]
